## Notebook for load and processing molecules  using rdkit

In this notebook we proceed to load the molecules (actives and decoys) from CSAR, DUD and DEKOIS2.0 datasets. Molecules are read from sdf files previously created.

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os, pickle

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')

### Useful functions 

In [3]:
from analyse_db_molecules import load_molecules_from_dir, get_files_list, get_mol_dataframe

## Cocristalized ligands
### Read the cocrystalized ligands
Previously, in notebook `1_Working_with_3D_structures_in_rdkit` we have saved a python object containing a dataframe with the cocristalized ligands. That dataframe is the following:

In [4]:
path_to_file_cocrys = os.path.join(*F'../data/rdkit_molecules/cdk2_365_cocristalized_ligands.pyobj'.split('/'))
with open(path_to_file_cocrys, 'rb') as f:
    df_crys_poses_rdk = pickle.load(f)
df_crys_poses_rdk.head()

,Lig,mol_rdk,file,Labels_conf,Validation
1aq1,STU,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b4b0>,../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS...,inact_b,v1
1b38,ATP,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b4f0>,../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS...,inact_a,v1
1b39,ATP,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b5b0>,../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS...,inact_a,v1
1ckp,PVB,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b5f0>,../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS...,inact_a,v1
1di8,DTQ,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b630>,../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS...,inact_a,v1


From this dataframe we are only interested in the ligand name, and in the `mol_rd` columns, which has the *rdkit* molecule instances. We also need to add column indicating that the molecule is an **Active**, or Inactive, ligand. Of course, because these molecules were cocristalized with the protein we're going to assume all of them are actives.  

Also, we are going to remove duplicate ligands, mainly because for this analysis we are not interested in the 3D conformation. **At the end we only going to keep 315 ligands from the original 365**.

In [5]:
df_COCRYS = df_crys_poses_rdk.reset_index()[['Lig', 'mol_rdk']]
df_COCRYS.drop_duplicates(subset = 'Lig', inplace = True)
df_COCRYS.set_index('Lig', inplace = True)
df_COCRYS['Activity'] = 'active'
df_COCRYS['sanitized'] = True
df_COCRYS = df_COCRYS[['Activity', 'mol_rdk', 'sanitized']]
df_COCRYS.shape

(315, 3)

In [6]:
df_COCRYS.head()

,Activity,mol_rdk,sanitized
Lig,,,
STU,active,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b4b0>,True
ATP,active,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b4f0>,True
PVB,active,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b5f0>,True
DTQ,active,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b630>,True
HMD,active,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b6b0>,True


## CSAR dataset
### Read the CSAR dataset molecules

In [7]:
# File of the python object
csar_obj_file = os.path.join(*F'../data/rdkit_molecules/cdk2_CSAR_ligands.pyobj'.split('/'))

if os.path.isfile(csar_obj_file):
    with open(csar_obj_file, 'rb') as f:
        df_CSAR = pickle.load(f)

else:
    # Firs we got the molecules dictionary
    path_to_file_csar_sdfs = \
    os.path.join(*F'../../ARCHIVOS/CRISTALES/LIGS_CDK2/CSAR/sdf/*'.split('/'))
    csar_file_list = glob(path_to_file_csar_sdfs)
    # sort alphanumericalñy
    csar_file_list.sort(key = lambda x: int(x.split('/')[-1].split('.')[0].replace('CS', '')))

    # Load the molecules using the function previously declared
    csar_molecules_dic = load_molecules_from_dir(csar_file_list)
    # Convert to a dataframe
    df_CSAR = pd.DataFrame(csar_molecules_dic).T
    df_CSAR.columns = ['mol_rdk', 'sanitized']
    # The information about if the molecule is active or inactive is in the following file
    df_csar_activity = pd.read_json('../data/csar_Binding_Data_Corrected_2016.json')
    # merge both dataframes
    df_CSAR['Activity'] = df_csar_activity['ActiveInactive'].str.lower()
    # Naming the columns
    df_CSAR = df_CSAR[['Activity', 'mol_rdk', 'sanitized']]
    
    # Save the file
    with open(csar_obj_file, 'wb') as f:
        pickle.dump(df_CSAR, f)

print(df_CSAR.shape)

(111, 3)


In [8]:
df_CSAR.head()

,Activity,mol_rdk,sanitized
CS1,active,<rdkit.Chem.rdchem.Mol object at 0x7f376a01b470>,True
CS2,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769f36970>,True
CS3,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769f369b0>,True
CS4,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769f369f0>,True
CS5,inactive,<rdkit.Chem.rdchem.Mol object at 0x7f3769f36a30>,True


## DUD 2006 Dataset

In [9]:
dud_obj_file = os.path.join(*F'../data/rdkit_molecules/cdk2_DUD_ligands.pyobj'.split('/'))

if os.path.isfile(dud_obj_file):
    with open(dud_obj_file, 'rb') as f:
        df_DUD = pickle.load(f)

else:
    # Firs we got the molecules dictionary
    path_to_file_dud_sdfs = \
    os.path.join(*F'../../ARCHIVOS/CRISTALES/LIGS_CDK2/DUDE_DECOYS/DUD_2006/sdf/'.split('/'))
    # Get the files list
    dud_file_list = get_files_list(path_to_sdfs = path_to_file_dud_sdfs, 
                                   actives_name='ligand', inactives_name='decoy')

    # Load the molecules using the function previously declared
    # load_molecules_from_dir(path_to_file_csar_sdfs)
    dud_molecules_dic = load_molecules_from_dir(dud_file_list)

    # Convert to a dataframe
    df_DUD = get_mol_dataframe(mol_dictionary=dud_molecules_dic)
    
    with open(dud_obj_file, 'wb') as f:
        pickle.dump(df_DUD, f)
        
print(df_DUD.shape)

(2146, 3)


In [10]:
df_DUD.head()

,Activity,mol_rdk,sanitized
ligand_1,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769f36930>,True
ligand_2,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ee93b0>,True
ligand_3,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ee93f0>,True
ligand_4,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ee9430>,True
ligand_5,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ee9470>,True


## DEKOIS Dataset

In [11]:
dekois_obj_file = os.path.join(*F'../data/rdkit_molecules/cdk2_DEKOIS_ligands.pyobj'.split('/'))

if os.path.isfile(dekois_obj_file):
    with open(dekois_obj_file, 'rb') as f:
        df_DEKOIS = pickle.load(f)

else:
    # Firs we got the molecules dictionary
    path_to_file_dekois_sdfs = \
    os.path.join(*F'../../ARCHIVOS/CRISTALES/LIGS_CDK2/DEKOIS_2/sdf/'.split('/'))
    # Get the files list
    dekois_file_list = get_files_list(path_to_sdfs = path_to_file_dekois_sdfs, 
                                   actives_name='ligand', inactives_name='decoy')

    # Load the molecules using the function previously declared
    # load_molecules_from_dir(path_to_file_csar_sdfs)
    dekois_molecules_dic = load_molecules_from_dir(dekois_file_list)

    # Convert to a dataframe
    df_DEKOIS = get_mol_dataframe(mol_dictionary=dekois_molecules_dic)
    
    with open(dekois_obj_file, 'wb') as f:
        pickle.dump(df_DEKOIS, f)
        
print(df_DEKOIS.shape)

(1240, 3)


In [12]:
df_DEKOIS.head()

,Activity,mol_rdk,sanitized
ligand_1,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ee9370>,True
ligand_2,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ec9bf0>,True
ligand_3,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ec9c30>,True
ligand_4,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ec9c70>,True
ligand_5,active,<rdkit.Chem.rdchem.Mol object at 0x7f3769ec9cb0>,True


In [23]:
df_DEKOIS.sanitized.sum()

1240

## DUD-E Dataset

In [28]:
dud_obj_file = os.path.join(*F'../data/rdkit_molecules/cdk2_DUDE_ligands.pyobj'.split('/'))

if os.path.isfile(dud_obj_file):
    with open(dud_obj_file, 'rb') as f:
        df_DUDE = pickle.load(f)

else:
    # Firs we got the molecules dictionary
    path_to_file_dud_sdfs = \
    os.path.join(*F'../../ARCHIVOS/CRISTALES/LIGS_CDK2/DUD_E/sdf/'.split('/'))
    # Get the files list
    dude_file_list = get_files_list(path_to_sdfs = path_to_file_dud_sdfs, 
                                   actives_name='ligand', inactives_name='decoy')

    # Load the molecules using the function previously declared
    # load_molecules_from_dir(path_to_file_csar_sdfs)
    dude_molecules_dic = load_molecules_from_dir(dude_file_list)

    # Convert to a dataframe
    df_DUDE = get_mol_dataframe(mol_dictionary=dude_molecules_dic)
    
    with open(dud_obj_file, 'wb') as f:
        pickle.dump(df_DUDE, f)
        
print(df_DUDE.shape)

(29126, 3)


In [29]:
df_DUDE.head()
df_DUDE.sanitized.sum()

29126

## Dictionary of Datasets

In [31]:
lig_datasets = {'COCRYS': df_COCRYS, 'CSAR': df_CSAR, 'DUD': df_DUD, 'DUDE': df_DUDE, 'DEKOIS': df_DEKOIS}

### Save the python dictionary using pickle

In [16]:
import pickle

file_rd_mols = './rdKit_db_molecules.obj'
if os.path.isfile(file_rd_mols):
    with open(file_rd_mols, 'rb') as f:
        lig_datasets = pickle.load(f)
else:
    with open(file_rd_mols, 'wb') as f:
        pickle.dump(lig_datasets, f)

In [32]:
import pickle

file_rd_mols = './rdKit_db_molecules_with_DUDE.obj'
if os.path.isfile(file_rd_mols):
    with open(file_rd_mols, 'rb') as f:
        lig_datasets = pickle.load(f)
else:
    with open(file_rd_mols, 'wb') as f:
        pickle.dump(lig_datasets, f)